<a href="https://colab.research.google.com/github/mhuertascompany/ULL_2022/blob/main/project/ULL22_MLproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Do not forget to select a GPU Runtime before starting. 

The goal of this project is to perform simulation based inference with deep learning to constrain cosmological parameters. We will use the [CAMELS project](https://www.camel-simulations.org/). The Cosmology and Astrophysics with Machine Learning Simulations (CAMELS) is a suite of cosmological simulations with varying comsologies and subgrid physics, specially desgined for training Machine Learnning models. 

For this exercice we are going to use the [Multifield dataset](https://camels-multifield-dataset.readthedocs.io/en/latest/), which consists of a series of 2D Maps of different quantities (Stellar Mass, Total Mass, Gas Temperature, Velocity ...) for different cosmologies and subgrid parameters. 

**You will present the results in a pdf document together with a link to the code you used to generate the different plots.**

# Connect to GCloud

Just run these cells. Do not modify.

In [ ]:
from google.colab import auth
auth.authenticate_user()
!gcloud config set project candels-270716

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

In [ ]:
!mkdir data
!gcsfuse --implicit-dirs -o allow_other -file-mode=777 -dir-mode=777 camels data


# Import python modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
import tensorflow as tf
import tensorflow_probability as tfp
import sklearn

tfd = tfp.distributions
tfpl = tfp.layers
tfk = tf.keras
tfkl = tf.keras.layers
plt.style.use(astropy_mpl_style)
print(tfp.__version__)

#Load Maps and Corresponding parameters
The maps can be changed.

The models can be: 
 - IllustrisTNG
 -  SIMBA

The map types can be:
-  T (Gas Temperature)
-  Mstar (stellar mass)
- Z (Metallicity) 
-  Mgas (Gas Mass)
- Vgas (Gas Velocity)
- Mtot (total mass) etc..

THe list of available files is in /content/data

In [ ]:
## This function loads the maps and corresponding parameters

def load_maps(model,maptype):

  fmaps = '/content/data/Maps_'+maptype+'_'+model+'_LH_z=0.00.npy'
  maps  = np.load(fmaps)
  fparams = '/content/data/params_'+model+'.txt'
  params  = np.loadtxt(fparams)
  return maps,params

  

In [ ]:
# loads the temperature map of Illustris TNG - can take a while
maps, params = load_maps('IllustrisTNG','T')

The following cell shows a random example of the loaded map together with the corresponding parameters. You can run it several times. The goal is to design a deep learning model to infer cosmological and astrophysical parameters using the different maps.

In [ ]:
from random import randrange
map_number = randrange(15000)
plt.imshow(np.log10(maps[map_number]),cmap=plt.get_cmap('binary_r'), origin='lower', interpolation='bicubic')
plt.show()
params_map = params[map_number//15] ## This is how params and maps are connected. 
print('Value of the parameters for this map')
print('Omega_m: %.5f'%params_map[0])
print('sigma_8: %.5f'%params_map[1])
print('A_SN1:   %.5f'%params_map[2])
print('A_AGN1:  %.5f'%params_map[3])
print('A_SN2:   %.5f'%params_map[4])
print('A_AGN2:  %.5f'%params_map[5])

# Exercice 1: 
Create a Convolutional Neural Network that takes as input the map of gas temperature (T) from the IllustrisTNG model loaded in the and estimates $\Omega_m$, the Dark Matter Density. The output of the Neural Network should be a Gaussian Probability Density Function. You can also experiment with multimodal Gaussians (see build_model() function defined below).

You will produce:
- A summary of the architecture used. 

     --> For this see plot_model() function from keras.utils.vis_utils and visualkeras libraries.
     
     
- A plot of the learning history including the trainng and the validation sets.

     --> You can use ideas from previous exercises (epochs vs. Loss).
     
     
- A plot of input vs. output on the test set, including error bars. You will use the mean of posterior as output.

     --> Real vs mean prediction for the test set, plot the identity function to compare.
     
     
- Ten example plots of posterior distributions, together with the ground truth.


- 1-sigma value of the posterior versus the true error for the test set. Does the posterior properly capture uncertainty? Comment.

In [ ]:
#define negative log likelihood loss

negloglik = lambda y, p_y: -p_y.log_prob(y)

def build_model(nfilters,num_components,input_shape,output_shape):
  cnn = tfk.Sequential([
  tfkl.Conv2D(
      nfilters, (4,4),
      input_shape=(input_shape,input_shape,1),
      padding="same",
      activation='relu'),
  tfkl.BatchNormalization(),
  tfkl.MaxPool2D((2,2),strides=2),
  tfkl.Conv2D(
      nfilters*2, (3,3),
      padding="same",
      activation='relu'),
  tfkl.MaxPool2D((2,2),strides=2),
  tfkl.Conv2D(
      nfilters*4, (2,2),
      padding="same",
      activation='relu'),
  tfkl.MaxPool2D((2,2),strides=2),    
  
  tf.keras.layers.Flatten(),      
  tfkl.Dense(128, activation='relu'),
  tfkl.Dense(64, activation='relu'),
  tfkl.Dense(64, activation='tanh'),
  tfkl.Dense(tfpl.MixtureNormal.params_size(num_components),activation=None),
############################################################################################
#### Only one of the next two lines should be uncommented, keep the other one commented ####
#### The "Dense" layer will output a probability per item (value between 0 & 1), while  ####
#### the "MixtureNormal" will output a probability density function                     ####
############################################################################################       
  #tfkl.Dense(1,activation='sigmoid')
  tfpl.MixtureNormal(num_components)
    ])     


  cnn.compile(optimizer=tf.optimizers.Adam(learning_rate=0.00002),loss=negloglik)

  return cnn

In [ ]:
Xtrain,Ytrain,Xval,Yval,Xtest,Ytest = train_test(3000,300,300)

In [ ]:
model1 = build_model(16,1,256,1)

# Exercice 2: 
Test the deep learning model on the same map, produced now with the SIMBA cosmological model. 

Tip: if you run out of memory, save the weights of the previous training (model.save_weights) in your GDrive , restart the notebook and load. GDrive can be mounted with the left menu.

- Produce a scatter plot, including error bars, of input Omega_m versus output Omega_m. Comment.
- Plot ten example plots of posterior distributions
- Plot the 1-sigma value of the posterior versus the true error. Does the posterior properly capture uncertainty? Comment.

# Exercice 3: 
Repeat the steps above with an astrophysics parameter, e.g A_AGN_1, which parametrizes the type of AGN feedback. Compare and comment.

# Exercice 4 [Optional]

Desing a domain adaptation technique so that the model trained on TNG can work in SIMBA.